__Probing Language Models__

This notebook serves as a start for your NLP2 assignment on probing Language Models. This notebook will become part of the contents that you will submit at the end, so make sure to keep your code (somewhat) clean :-)

__note__: This is the first time _anyone_ is doing this assignment. That's exciting! But it might well be the case that certain aspects are too unclear. Do not hesitate at all to reach to me once you get stuck, I'd be grateful to help you out.

__note 2__: This notebook only contains the setup for the POS probing experiment. The structural probing setup will follow soon, but I'm still working out some things there. This should keep you occupied for now hopefully, good luck!

__note 3__: This assignment is not dependent on big fancy GPUs. I run all this stuff on my own 3 year old CPU, without any Colab hassle. So it's up to you to decide how you want to run it.

# Models

For the Transformer models you are advised to make use of the `transformers` library of Huggingface: https://github.com/huggingface/transformers
Their library is well documented, and they provide great tools to easily load in pre-trained models.

In [ ]:
#
## Your code for initializing the transformer model(s)
#
# Note that most transformer models use their own `tokenizer`, that should be loaded in as well.
#
from transformers import *


model_TF = GPT2Model.from_pretrained('distilgpt2', output_hidden_states = True)
tokenizer_TF = GPT2Tokenizer.from_pretrained('distilgpt2')


# Note that some models don't return the hidden states by default.
# This can be configured using a `PretrainedConfig` object, or directly like:
# model.transformer.output_hidden_states = True

In [ ]:
#
## Your code for initializing the rnn model(s)
#
# The Gulordava LSTM model can be found here: 
# https://github.com/facebookresearch/colorlessgreenRNNs/tree/master/data
#
from collections import defaultdict
from model import RNNModel
import torch


model_location = 'RNN/Gulordava.pt'  # <- point this to the location of the Gulordava .pt file
device = 'cpu'
rnn = torch.load(model_location, map_location=device)


# This LSTM does not use a Tokenizer like the Transformers, but a Vocab dictionary that maps a token to an id.
with open('RNN/vocab.txt') as f:
    w2i = {w.strip(): i for i, w in enumerate(f)}

vocab_dict = defaultdict(lambda: w2i["<unk>"])
vocab_dict.update(w2i)

# Data

For this assignment you will train your probes on __treebank__ corpora. A treebank is a corpus that has been *parsed*, and stored in a representation that allows the parse tree to be recovered. Next to a parse tree, treebanks also often contain information about part-of-speech tags, which is exactly what we are after now.

The treebank you will use for now is part of the Universal Dependencies project. I provide a sample of this treebank as well, so you can test your setup on that before moving on to larger amounts of data.

Make sure you accustom yourself to the format that is created by the `conllu` library that parses the treebank files before moving on. For example, make sure you understand how you can access the pos tag of a token, or how to cope with the tree structure that is formed using the `to_tree()` functionality.

In [ ]:
# READ DATA
from typing import List
from conllu import parse_incr, TokenList


# If stuff like `: str` and `-> ..` seems scary, fear not! 
# These are type hints that help you to understand what kind of argument and output is expected.
def parse_corpus(filename: str) -> List[TokenList]:
    data_file = open(filename, encoding="utf-8")

    ud_parses = list(parse_incr(data_file))
    
    return ud_parses

# Generating Representations

We now have our data all set, our models are running and we are good to go!

The next step is now to create the model representations for the sentences in our corpora. Once we have generated these representations we can store them, and train additional diagnostic (/probing) classifiers on top of the representations.

There are a few things you should keep in mind here. Read these carefully, as these tips will save you a lot of time in your implementation.
- Transformer models make use of Byte-Pair Encodings (BPE), that chunk up a piece of next in subword pieces. For example, a word such as "largely" could be chunked up into "large" and "ly". We are interested in probing linguistic information on the __word__-level. Therefore, we will follow the suggestion of Hewitt et al. (2019a, footnote 4), and create the representation of a word by averaging over the representations of its subwords. So the representation of "largely" becomes the average of that of "large" and "ly".

- Subword chunks never overlap multiple tokens. In other words, say we have a phrase like "None of the", then the tokenizer might chunk that into "No"+"ne"+" of"+" the", but __not__ into "No"+"ne o"+"f the", as those chunks overlap multiple tokens. This is great for our setup! Otherwise it would have been quite challenging to distribute the representation of a subword over the 2 tokens it belongs to.

- If you closely examine the provided treebank, you will notice that some tokens are split up into multiple pieces, that each have their own POS-tag. For example, in the first sentence the word "Al-Zaman" is split into "Al", "-", and "Zaman". In such cases, the conllu `TokenList` format will add the following attribute: `('misc', OrderedDict([('SpaceAfter', 'No')]))` to these tokens. Your model's tokenizer does not need to adhere to the same tokenization. E.g., "Al-Zaman" could be split into "Al-"+"Za"+"man", making it hard to match the representations with their correct pos-tag. Therefore I recommend you to not tokenize your entire sentence at once, but to do this based on the chunking of the treebank. Make sure to still incoporate the spaces in a sentence though, as these are part of the BPE of the tokenizer. The tokenizer for GPT-2 adds spaces at the start of a token.

- The LSTM LM does not have the issues related to subwords, but is far more restricted in its vocabulary. Make sure you keep the above points in mind though, when creating the LSTM representations. You might want to write separate functions for the LSTM, but that is up to you.

I would like to stress that if you feel hindered in any way by the simple code structure that is presented here, you are free to modify it :-) Just make sure it is clear to an outsider what you're doing, some helpful comments never hurt.

In [ ]:
# FETCH SENTENCE REPRESENTATIONS transformer
from torch import Tensor
from tqdm import tqdm


# Should return a tensor of shape (num_tokens_in_corpus, representation_size)
# Make sure you correctly average the subword representations that belong to 1 token!
def fetch_sen_reps(ud_parses: List[TokenList], model, tokenizer, model_type) -> Tensor:
    sen_reps = []
    for sentence in tqdm(ud_parses):
        for word in sentence:
            if model_type=='TF':
                input_ids = Tensor(tokenizer.encode(word['form'])).type(torch.long).unsqueeze(0)
                output = model(input_ids)[0][0].mean(dim=0).detach()
            elif model_type=='RNN':
                input_ids = Tensor([tokenizer[word['form']]]).type(torch.long)
                output = model.encoder(input_ids).squeeze().detach()
                
            sen_reps.append(output)
    
    return torch.stack(sen_reps)

In [ ]:
# FETCH POS LABELS transformer


# Should return a tensor of shape (num_tokens_in_corpus,)
# Make sure that when fetching these pos tags for your train/dev/test corpora you share the label vocabulary.
def fetch_pos_tags(ud_parses: List[TokenList], vocab=None) -> Tensor:
    pos_tags = []
    vocab_list = []
    for sentence in ud_parses:
        for word in sentence:
            pos_tag = word['upostag']
            
            if vocab:
                if pos_tag not in vocab:
                    pos_tags.append(vocab['unk'])
                
                else:
                    pos_tags.append(vocab[pos_tag])
            
            else:
                if pos_tag not in vocab_list:
                    vocab_list.append(pos_tag)
                
                pos_tags.append(vocab_list.index(pos_tag))
    
    if not vocab:
        vocab_list.append('unk')
        vocab_list = {vocab_list[i]:i for i in range(len(vocab_list))}
    
    return Tensor(pos_tags), vocab_list

In [ ]:
import os

# Function that combines the previous functions, and creates 2 tensors for a .conllu file: 
# 1 containing the token representations, and 1 containing the (tokenized) pos_tags.

def create_data(filename: str, model, tokenizer, vocab=None, model_type='TF'):
    ud_parses = parse_corpus(filename)
    
    sen_reps = fetch_sen_reps(ud_parses, model, tokenizer, model_type)
    
    pos_tags, pos_vocab = fetch_pos_tags(ud_parses, vocab=vocab)
    
    return sen_reps, pos_tags, pos_vocab


use_sample = True

data = {'TF':{'train':[], 'dev':[], 'test':[]}, 'RNN':{'train':[], 'dev':[], 'test':[]}}
tokenizer = {'TF':tokenizer_TF, 'RNN':vocab_dict}
model = {'TF':model_TF, 'RNN':rnn}

for i, model_type in enumerate(data):
    train_x, train_y, train_vocab = create_data(
        os.path.join('data', 'sample' if use_sample else '', 'en_ewt-ud-train.conllu'),
        model[model_type], 
        tokenizer[model_type],
        model_type=model_type
    )
    data[model_type]['train'].append(train_x)
    data[model_type]['train'].append(train_y)

    dev_x, dev_y, _ = create_data(
        os.path.join('data', 'sample' if use_sample else '', 'en_ewt-ud-dev.conllu'),
        model[model_type], 
        tokenizer[model_type],
        vocab=train_vocab,
        model_type=model_type
    )
    data[model_type]['dev'].append(dev_x)
    data[model_type]['dev'].append(dev_y)

    test_x, test_y, _ = create_data(
        os.path.join('data', 'sample' if use_sample else '', 'en_ewt-ud-test.conllu'),
        model[model_type],
        tokenizer[model_type],
        vocab=train_vocab,
        model_type=model_type
    )
    data[model_type]['test'].append(test_x)
    data[model_type]['test'].append(test_y)


torch.save(data, 'distilgpt2-rnn-pos.pt')

# Diagnostic Classification

We now have our models, our data, _and_ our representations all set! Hurray, well done. We can finally move onto the cool stuff, i.e. training the diagnostic classifiers (DCs).

DCs are purposefully simple in their complexity. To read more about why this is the case you could already have a look at the "Designing and Interpreting Probes with Control Tasks" by Hewitt and Liang (esp. Sec. 3.2).

A simple linear classifier will suffice for now, don't bother with adding fancy non-linearities to it.

I am personally a fan of the `skorch` library, that provides `sklearn`-like functionalities for training `torch` models, but you are free to train your dc using whatever method you prefer.

As this is an Artificial Intelligence master and you have all done ML1 + DL, I expect you to use your train/dev/test splits correctly ;-)

In [ ]:
# DIAGNOSTIC CLASSIFIER
import torch.nn as nn
import numpy as np

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class DC(nn.Module):
    def __init__(self, features, pos_tags, layers=[]):
        super(DC, self).__init__()

        self.neg_slope = 0.01

        layers = [features] + layers + [pos_tags]
        modules = []
        for i in range(len(layers)-1):
            modules.append(nn.Linear(layers[i], layers[i+1]))
            if i>0:
                modules.append(nn.LeakyReLU(self.neg_slope))
        
        self.net = nn.Sequential(*modules)
    
    def forward(self, x):
        out = self.net(x)

        return out
    

epochs = 300
batch_size = 64
learning_rate = 1e-3

models = {}

loss = nn.CrossEntropyLoss()

for model_type in data:
    train_x = data[model_type]['train'][0]
    train_y = data[model_type]['train'][1]
    dev_x = data[model_type]['dev'][0]
    dev_y = data[model_type]['dev'][1]
    test_x = data[model_type]['test'][0]
    test_y = data[model_type]['test'][1]

    model = DC(train_x.size()[1], len(train_vocab)).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    total_batches = int(len(train_x)/batch_size)+1    
    total_batches_dev = int(len(dev_x)/batch_size)+1
    total_batches_test = int(len(test_x)/batch_size)+1
    min_loss = np.inf
    patience = 0
    max_patience = 5
    
    print("\n---------------------------------------------------------")
    print("Model: " + model_type)
    print("---------------------------------------------------------")
    
    for epoch in range(epochs):
        losses = []
        for i in range(total_batches):
            if i < total_batches-1:
                features = train_x[i*batch_size:(i+1)*batch_size].to(dtype=torch.float, device=device)
                labels = train_y[i*batch_size:(i+1)*batch_size].to(dtype=torch.long, device=device)
            else:
                features = train_x[i*batch_size:].to(dtype=torch.float, device=device)
                labels = train_y[i*batch_size:].to(dtype=torch.long, device=device)


            preds = model(features)
            loss_out = loss(preds, labels)

            optimizer.zero_grad()
            loss_out.backward()
            optimizer.step()
            losses.append(loss_out.item())
            
        loss_train = np.mean(losses)
        
        print("\n---------------------------------------------------------")
        print("epoch: " + str(epoch+1))
        print("training set loss: " + str(loss_train))
        
        with torch.no_grad():
            losses = []
            for i in range(total_batches_dev):
                if i < total_batches_dev-1:
                    features = dev_x[i*batch_size:(i+1)*batch_size].to(dtype=torch.float, device=device)
                    labels = dev_y[i*batch_size:(i+1)*batch_size].to(dtype=torch.long, device=device)
                else:
                    features = dev_x[i*batch_size:].to(dtype=torch.float, device=device)
                    labels = dev_y[i*batch_size:].to(dtype=torch.long, device=device)

                preds = model(features)
                loss_out = loss(preds, labels)
                losses.append(loss_out.item())
            
            loss_dev = np.mean(losses)
            
            #Early Stopping
            if loss_dev>min_loss:
                patience+=1
            else:
                min_loss = loss_dev
                patience = 0

            print("validation set loss: " + str(loss_dev) + ", patience: " + str(patience))
            print("---------------------------------------------------------")

            if patience==max_patience:
                print("\n---------------------------------------------------------")
                print("Early Stopping at epoch:", str(epoch+1))
                break
                
        models[model_type] = model
                
    with torch.no_grad():
        losses = []
        for i in range(total_batches_test):
            if i < total_batches_test-1:
                features = test_x[i*batch_size:(i+1)*batch_size].to(dtype=torch.float, device=device)
                labels = test_y[i*batch_size:(i+1)*batch_size].to(dtype=torch.long, device=device)
            else:
                features = test_x[i*batch_size:].to(dtype=torch.float, device=device)
                labels = test_y[i*batch_size:].to(dtype=torch.long, device=device)
            
            preds = model(features)
            loss_out = loss(preds, labels)
            losses.append(loss_out.item())

        loss_test = np.mean(losses)
        print("test set loss: " + str(loss_test) + ", model-type: " + model_type)
        print("---------------------------------------------------------")
        
